# Customization and Rate-Limited Concurrency

In [1]:
import lionagi as li
from timeit import default_timer as timer

let us use a simple conditional calculator session as an example

in this example, we will have two steps in the instruction, first step would be choosing between sum or diff based on a case number

and second step would be choosing between times or plus based on the sign of the first step

In [2]:
system = """
You are asked to perform as a calculator. Return only a numeric 
value, i.e. int or float, no text.
"""

instruction1 = """
sum the absolute values, provided with 2 numbers, return the sum
of their absolute values. i.e. |x|+|y|",
"""

instruction2 = """
diff the absolute values, provided with 2 numbers, return the 
difference of absolute values. i.e. |x|-|y|
"""

instruction3 = """
if previous response is positive, times 2. i.e. *2,
else, plus 2. i.e. +2
"""

In [3]:
# create a case and context
case = 0
context = {"x": 7, "y": 3}
instruct = instruction1 if case == 0 else instruction2

In [4]:
start = timer()
calculator = li.Branch(system=system)

step1 = await calculator.chat(instruct, context=context)
step2 = await calculator.chat(
    instruction3, temperature=0.5
)  # you can also modify parameters for each API call

print(f"step1 result: {step1}")
print(f"step2 result: {step2}")

elapsed_time = timer() - start
print(f"run clock time: {elapsed_time:0.2f} seconds")

step1 result: 10
step2 result: 20
run clock time: 1.50 seconds


In [5]:
# now let us run 10 senerios in parallel
import numpy as np

num_iterations = 10

In [6]:
# generate random numbers
ints1 = np.random.randint(-10, 10, size=num_iterations)
ints2 = np.random.randint(0, 10, size=num_iterations)
cases = np.random.randint(0, 2, size=num_iterations)


contexts = []
for i in range(num_iterations):
    contexts.append({"x": str(ints1[i]), "y": str(ints2[i]), "case": str(cases[i])})

    print(contexts[i])

{'x': '5', 'y': '2', 'case': '1'}
{'x': '8', 'y': '5', 'case': '0'}
{'x': '-2', 'y': '8', 'case': '0'}
{'x': '-4', 'y': '5', 'case': '1'}
{'x': '-2', 'y': '4', 'case': '0'}
{'x': '-4', 'y': '4', 'case': '0'}
{'x': '-3', 'y': '8', 'case': '1'}
{'x': '4', 'y': '7', 'case': '1'}
{'x': '-5', 'y': '8', 'case': '0'}
{'x': '-3', 'y': '3', 'case': '0'}


In [7]:
async def calculator_workflow(context):

    calculator = li.Branch(system=system)  # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))

    instruct = instruction1 if case == 0 else instruction2
    res1 = await calculator.chat(instruct, context=context)  # run the steps
    res2 = await calculator.chat(instruction3, temperature=0.5)

    return (res1, res2)

In [8]:
start = timer()
outs = await li.alcall(contexts, calculator_workflow)
elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 10
run clock time: 1.04 seconds


In [9]:
outs

[(3, 6),
 (13, 26),
 (10, 20),
 (1, 2),
 (6, 12),
 (8, 16),
 (5, 10),
 (3, 6),
 (13, 26),
 (6, 12)]

## Customized API service concurrent calls

By default, all the session will initiate a new default service object.

If you would like to have one service using across sessions, you need to pass in the **same** service object during construction.

In [18]:
imodel = li.iModel(
    model="gpt-4o",
    interval_tokens=1000,
    interval_requests=3,
    interval=5,
    temperature=0.5,
)


async def calculator_workflow(context):

    calculator = li.Branch(system, imodel=imodel)  # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))
    instruct = instruction1 if case == 0 else instruction2

    res1 = await calculator.chat(instruct, context=context)  # run the steps
    res2 = await calculator.chat(instruction3)

    return (res1, res2)

In [19]:
start = timer()

outs = await li.alcall(contexts, calculator_workflow)

elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 10
run clock time: 8.38 seconds


In [17]:
outs

[(3, 6),
 (13, 26),
 (10, 20),
 (1, 2),
 (6, 12),
 (8, 16),
 (5, 10),
 (3, 6),
 (13, 26),
 (6, 12)]